In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv("D:/CB Git/machine-learning-online-2018/13. Decision Trees/titanic.csv")

In [3]:
data.head(n=5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
columns_to_drop = ["PassengerId","Name","Ticket","Cabin","Embarked"]

data_clean = data.drop(columns_to_drop,axis=1)

In [6]:
data_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
dtypes: float64(2), int64(4), object(1)
memory usage: 48.9+ KB


In [7]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data_clean["Sex"] = le.fit_transform(data_clean["Sex"])

In [8]:
data_clean.head(n=5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,1,22.0,1,0,7.2500
1,1,1,0,38.0,1,0,71.2833
2,1,3,0,26.0,0,0,7.9250
3,1,1,0,35.0,1,0,53.1000
4,0,3,1,35.0,0,0,8.0500


In [9]:
data_clean = data_clean.fillna(data_clean["Age"].mean())

In [32]:
data_clean.head(n=5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,1,22.0,1,0,7.2500
1,1,1,0,38.0,1,0,71.2833
2,1,3,0,26.0,0,0,7.9250
3,1,1,0,35.0,1,0,53.1000
4,0,3,1,35.0,0,0,8.0500


In [10]:
data_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    int32  
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
dtypes: float64(2), int32(1), int64(4)
memory usage: 45.4 KB


Entropy and Information gain

In [11]:
input_cols = ["Pclass","Sex","Age","SibSp","Parch","Fare"]
output_cols = ["Survived"]

X = data_clean[input_cols]
Y = data_clean[output_cols]

print(X.shape,Y.shape)

(891, 6) (891, 1)


In [12]:
def entropy(col):
    counts = np.unique(col,return_counts=True)
    N = float(col.shape[0])
    
    ent = 0.0
    
    for i in counts[1]:
        p = i/N
        ent += (-1.0*p*np.log2(p))
        
    return ent

In [13]:
def divide_data(x_data,fkey,fval):
    x_right = pd.DataFrame([],columns=x_data.columns)
    x_left = pd.DataFrame([],columns=x_data.columns)
    
    for ix in range(x_data.shape[0]):
        val = x_data[fkey].loc[ix]
        
        if val > fval:
            x_right = x_right.append(x_data.loc[ix])
        else:
            x_left = x_left.append(x_data.loc[ix])
            
    return x_left,x_right

In [14]:
def information_gain(x_data,fkey,fval):
    
    left,right = divide_data(x_data,fkey,fval)
    
    #Calculating % of samples on left and right side
    l = float(left.shape[0]/x_data.shape[0])
    r = float(right.shape[0]/x_data.shape[0])
    
    #If all examples come to one side i.e no further split is possible
    if left.shape[0] == 0 or right.shape[0] == 0:
        return -1000000
    
    i_gain = entropy(x_data.Survived) - (l*entropy(left.Survived) + r*entropy(right.Survived))
    return i_gain

In [15]:
#Test our function
for fx in X.columns:
    print(fx)
    print(information_gain(data_clean,fx,data_clean[fx].mean()))

Pclass
0.07579362743608165
Sex
0.2176601066606142
Age
0.0008836151229467681
SibSp
0.009584541813400071
Parch
0.015380754493137694
Fare
0.042140692838995464


In [16]:
#Decision Tree
class DecisionTree:
    
    #Constructor
    def __init__(self,depth=0,max_depth=5):
        self.left = None
        self.right = None
        self.fkey = None
        self.fval = None
        self.depth = depth
        self.max_depth = max_depth
        self.target = None
        
    def train(self,X_train):
        features = ["Pclass","Sex","Age","SibSp","Parch","Fare"]
        info_gains = []
        
        for ix in features:
            i_gain = information_gain(X_train,ix,X_train[ix].mean())
            info_gains.append(i_gain)
            
        self.fkey = features[np.argmax(info_gains)]
        self.fval = X_train[self.fkey].mean()
        print("Making Tree feature", self.fkey)
        
        #Split
        data_left, data_right = divide_data(X_train,self.fkey,self.fval)
        data_left = data_left.reset_index(drop=True)
        data_right = data_right.reset_index(drop=True)
        
        #Truly a leaf node
        if data_left.shape[0] == 0 or data_right.shape[0] == 0:
            if X_train.Survived.mean() >= 0.5:
                self.target = "Survive"
            else:
                self.target = "Dead"
            return
                
        #Stop early when depth >= max_depth
        if self.depth >= self.max_depth:
            if X_train.Survived.mean() >= 0.5:
                self.target = "Survive"
            else:
                self.target = "Dead"
            return
        
        #Recursive Case
        self.left = DecisionTree(depth=self.depth+1,max_depth=self.max_depth)
        self.left.train(data_left)
        
        self.right = DecisionTree(depth=self.depth+1,max_depth=self.max_depth)
        self.right.train(data_right)
        
        #You can set the target at every node
        if X_train.Survived.mean() >= 0.5:
                self.target = "Survive"
        else:
                self.target = "Dead"
        return
    
    def predict(self,test):
        if test[self.fkey] >= self.fval:
            #go to right
            if self.right is None:
                return self.target
            return self.right.predict(test)
        else:
            if self.left is None:
                return self.target
            return self.left.predict(test)

In [17]:
#Test data

In [42]:
test_data = pd.read_csv("C:/Users/91768/Downloads/Datasets/Test.csv")

In [43]:
print(test_data.shape)

(300, 13)


In [44]:
test_data.head(n=5)

,Pclass,name,Sex,Age,SibSp,Parch,ticket,Fare,cabin,embarked,boat,body,home.dest
0,1,"Flynn, Mr. John Irwin (""Irving"")",male,36.0,0,0,PC 17474,26.3875,E25,S,5,NaN,"Brooklyn, NY"
1,3,"Sage, Miss. Constance Gladys",female,NaN,8,2,CA. 2343,69.5500,NaN,S,NaN,NaN,NaN
2,1,"Rood, Mr. Hugh Roscoe",male,NaN,0,0,113767,50.0000,A32,S,NaN,NaN,"Seattle, WA"
3,2,"Gillespie, Mr. William Henry",male,34.0,0,0,12233,13.0000,NaN,S,NaN,NaN,"Vancouver, BC"
4,2,"Collander, Mr. Erik Gustaf",male,28.0,0,0,248740,13.0000,NaN,S,NaN,NaN,"Helsinki, Finland Ashtabula, Ohio"


In [74]:
test_columns_to_drop = ["boat","body","name","ticket","cabin","embarked","home.dest"]

test_data_clean = test_data.drop(test_columns_to_drop,axis=1)

In [75]:
test_data_clean.head(n=5)

,Pclass,Sex,Age,SibSp,Parch,Fare
0,1,male,36.0,0,0,26.3875
1,3,female,NaN,8,2,69.5500
2,1,male,NaN,0,0,50.0000
3,2,male,34.0,0,0,13.0000
4,2,male,28.0,0,0,13.0000


In [76]:
test_data_clean["Sex"] = le.fit_transform(test_data_clean["Sex"])

In [77]:
test_data_clean.head(n=5)

,Pclass,Sex,Age,SibSp,Parch,Fare
0,1,1,36.0,0,0,26.3875
1,3,0,NaN,8,2,69.5500
2,1,1,NaN,0,0,50.0000
3,2,1,34.0,0,0,13.0000
4,2,1,28.0,0,0,13.0000


In [78]:
test_data_clean = test_data_clean.fillna(test_data_clean["Age"].mean())

In [79]:
test_data_clean.head(n=5)

,Pclass,Sex,Age,SibSp,Parch,Fare
0,1,1,36.000000,0,0,26.3875
1,3,0,30.027422,8,2,69.5500
2,1,1,30.027422,0,0,50.0000
3,2,1,34.000000,0,0,13.0000
4,2,1,28.000000,0,0,13.0000


In [81]:
test_data_clean["Age"] = test_data_clean["Age"].astype(float)

In [82]:
test_data_clean.head(n=5)

,Pclass,Sex,Age,SibSp,Parch,Fare
0,1,1,36.0,0,0,26.3875
1,3,0,30.0,8,2,69.5500
2,1,1,30.0,0,0,50.0000
3,2,1,34.0,0,0,13.0000
4,2,1,28.0,0,0,13.0000


In [83]:
test_data_clean["Survived"] = " "

In [84]:
test_data_clean.head(n=5)

,Pclass,Sex,Age,SibSp,Parch,Fare,Survived
0,1,1,36.0,0,0,26.3875,
1,3,0,30.0,8,2,69.5500,
2,1,1,30.0,0,0,50.0000,
3,2,1,34.0,0,0,13.0000,
4,2,1,28.0,0,0,13.0000,


In [85]:
test_data_clean = test_data_clean[["Survived","Pclass","Sex","Age","SibSp","Parch","Fare"]]

In [86]:
test_data_clean.head(n=5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,,1,1,36.0,0,0,26.3875
1,,3,0,30.0,8,2,69.5500
2,,1,1,30.0,0,0,50.0000
3,,2,1,34.0,0,0,13.0000
4,,2,1,28.0,0,0,13.0000


In [23]:
#Training the dataset

In [39]:
dt = DecisionTree()
dt.train(data_clean)

Making Tree feature Sex
Making Tree feature Pclass
Making Tree feature Pclass
Making Tree feature Parch
Making Tree feature Age
Making Tree feature Age
Making Tree feature Age
Making Tree feature Parch
Making Tree feature Age
Making Tree feature Fare
Making Tree feature Parch
Making Tree feature Age
Making Tree feature Age
Making Tree feature Age
Making Tree feature Age
Making Tree feature Age
Making Tree feature Age
Making Tree feature Fare
Making Tree feature SibSp
Making Tree feature Fare
Making Tree feature Fare
Making Tree feature Age
Making Tree feature SibSp
Making Tree feature Parch
Making Tree feature Age
Making Tree feature SibSp
Making Tree feature Fare
Making Tree feature Parch
Making Tree feature Parch
Making Tree feature Age
Making Tree feature Age
Making Tree feature Age
Making Tree feature Fare
Making Tree feature Parch
Making Tree feature Pclass
Making Tree feature Pclass
Making Tree feature Age
Making Tree feature SibSp
Making Tree feature Age
Making Tree feature Age


In [87]:
data_clean.head(n=5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,1,22.0,1,0,7.2500
1,1,1,0,38.0,1,0,71.2833
2,1,3,0,26.0,0,0,7.9250
3,1,1,0,35.0,1,0,53.1000
4,0,3,1,35.0,0,0,8.0500


In [88]:
test_data_clean.head(n=5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,,1,1,36.0,0,0,26.3875
1,,3,0,30.0,8,2,69.5500
2,,1,1,30.0,0,0,50.0000
3,,2,1,34.0,0,0,13.0000
4,,2,1,28.0,0,0,13.0000


In [89]:
print(data_clean.shape,test_data_clean.shape)

(891, 7) (300, 7)


In [90]:
y_pred = []

for ix in range(test_data_clean.shape[0]):
    y_pred.append(dt.predict(test_data_clean.loc[ix]))

In [91]:
y_pred

['Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survive',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survive',
 'Dead',
 'Dead',
 'Survive',
 'Survive',
 'Dead',
 'Survive',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survive',
 'Dead',
 'Survive',
 'Survive',
 'Dead',
 'Dead',
 'Survive',
 'Dead',
 'Survive',
 'Dead',
 'Survive',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survive',
 'Survive',
 'Dead',
 'Survive',
 'Dead',
 'Dead',
 'Survive',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survive',
 'Dead',
 'Survive',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survive',
 'Dead',
 'Survive',
 'Survive',
 'Survive',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survive',
 'Survive',
 'Dead',
 'Dead',
 'Dead',
 'Survive',
 'Dead',
 'Survive',
 'Survive',
 'Dead',
 'Dead',
 'Survive',
 'Dead',
 'Dead',
 'Survive',
 'Survive',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Dead',
 'Survive',
 'Dead',
 'Dead',
 'Dead',
 'Survive',
 'Survive',
 'Dead',
 'Survive',
 'Survive',
 'Dead'

In [102]:
y_pred1 = le.fit_transform(y_pred)

In [103]:
y_pred1

array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0,
       1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0,
       0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1])

In [93]:
y_pred.shape

AttributeError: 'list' object has no attribute 'shape'

In [104]:
y_pred1 = pd.Series(y_pred1)

In [105]:
type(y_pred1)

pandas.core.series.Series

In [106]:
y_prediction = pd.DataFrame(data = y_pred1, columns=["survived"])

In [107]:
y_prediction = y_prediction.rename_axis(index='Id', columns="survived")

In [108]:
y_prediction.to_csv('titanic_prediction.csv',index= True)